In [67]:
import numpy as np
class CustomDecisionTree:
  def __init__(self, max_depth=None):
    """
    Initializes the decision tree with the specified maximum depth.
    Parameters:
    max_depth (int, optional): The maximum depth of the tree. If None, the tree is expanded until all
    leaves are pure or contain fewer than the minimum samples required to split.
    """
    self.max_depth = max_depth
    self.tree = None
  def fit(self, X, y):
    """
    Trains the decision tree model using the provided training data.
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for training the model.
    y (array-like): Target labels (n_samples,) for training the model.
    """
    self.tree = self._build_tree(X, y)
  def _build_tree(self, X, y, depth=0):
    """
    Recursively builds the decision tree by splitting the data based on the best feature and threshold
    .
    Parameters:
    X (array-like): Feature matrix (n_samples, n_features) for splitting.
    y (array-like): Target labels (n_samples,) for splitting.
    depth (int, optional): Current depth of the tree during recursion.
    Returns:
    dict: A dictionary representing the structure of the tree, containing the best feature index,
    threshold, and recursive tree nodes.
    """
    num_samples, num_features = X.shape
    unique_classes = np.unique(y)
    # Stopping conditions: pure node or reached max depth
    if len(unique_classes) == 1:
      return {'class': unique_classes[0]}
    if num_samples == 0 or (self.max_depth and depth >= self.max_depth):
      return {'class': np.bincount(y).argmax()}
    # Find the best split based on Information Gain (using Entropy)
    best_info_gain = -float('inf')
    best_split = None
    for feature_idx in range(num_features):
      thresholds = np.unique(X[:, feature_idx])
      for threshold in thresholds:
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        left_y = y[left_mask]
        right_y = y[right_mask]
        info_gain = self._information_gain(y, left_y, right_y)
        if info_gain > best_info_gain:
          best_info_gain = info_gain
          best_split = {
            'feature_idx': feature_idx,
            'threshold': threshold,
            'left_y': left_y,
            'right_y': right_y,
    }

    if best_split is None:
      return {'class': np.bincount(y).argmax()}

    # Recursively build the left and right subtrees
    left_tree = self._build_tree(X[best_split['left_y']], best_split['left_y'], depth + 1)
    right_tree = self._build_tree(X[best_split['right_y']], best_split['right_y'], depth + 1)

    return {'feature_idx': best_split['feature_idx'], 'threshold': best_split['threshold'],'left_tree': left_tree, 'right_tree': right_tree}

  def _information_gain(self, parent, left, right):
    """
    Computes the Information Gain between the parent node and the left/right child nodes.
    Parameters:
    parent (array-like): The labels of the parent node.
    left (array-like): The labels of the left child node.
    right (array-like): The labels of the right child node.
    Returns:
    float: The Information Gain of the split.
    """
    parent_entropy = self._entropy(parent)
    left_entropy = self._entropy(left)
    right_entropy = self._entropy(right)
    # Information Gain = Entropy(parent) - (weighted average of left and right entropies)
    weighted_avg_entropy = (len(left) / len(parent)) * left_entropy + (len(right) / len(parent)) * right_entropy

    return parent_entropy - weighted_avg_entropy

# def _entropy(self, y):
#   """
#   Computes the entropy of a set of labels.
#   Parameters:
#   y (array-like): The labels for which entropy is calculated.
#   Returns:
#   float: The entropy of the labels.
#   """
#   # Calculate the probability of each class
#   class_probs = np.bincount(y) / len(y)
#   # Compute the entropy using the formula: -sum(p * log2(p))
#   return -np.sum(class_probs * np.log2(class_probs + 1e-9)) # Added small epsilon to avoid log(0)
# def predict(self, X):
#   """
#   Predicts the target labels for the given test data based on the trained decision tree.
#   Parameters:
#   X (array-like): Feature matrix (n_samples, n_features) for prediction.
#   Returns:
#   list: A list of predicted target labels (n_samples,).
#   """
#   return [self._predict_single(x, self.tree) for x in X]

# def _predict_single(self, x, tree):
#   """
#   Recursively predicts the target label for a single sample by traversing the tree.
#   Parameters:
#   x (array-like): A single feature vector for prediction.
#   tree (dict): The current subtree or node to evaluate.
#   Returns:
#   int: The predicted class label for the sample.
#   """
#   if 'class' in tree:
#     return tree['class']
#   feature_val = x[tree['feature_idx']]
#   if feature_val <= tree['threshold']:
#     return self._predict_single(x, tree['left_tree'])
#   else:
#     return self._predict_single(x, tree['right_tree'])

  def _entropy(self, y):
    """
    Computes the entropy of a set of labels.
    """
    class_probs = np.bincount(y) / len(y)
    return -np.sum(class_probs * np.log2(class_probs + 1e-9))

  def predict(self, X):
    """
    Predicts the target labels for the given test data.
    """
    return [self._predict_single(x, self.tree) for x in X]

  def _predict_single(self, x, tree):
    """
    Recursively predicts the target label for a single sample.
    """
    if 'class' in tree:
      return tree['class']
    if x[tree['feature_idx']] <= tree['threshold']:
      return self._predict_single(x, tree['left_tree'])
    else:
      return self._predict_single(x, tree['right_tree'])


In [68]:
# Necessary Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Load the Iris dataset
data = load_iris()
X = data.data
y = data.target
# Split into training and test sets (80% training, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the custom decision tree
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_custom = custom_tree.predict(X_test)
# Calculate accuracy
accuracy_custom = accuracy_score(y_test, y_pred_custom)
print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")

Custom Decision Tree Accuracy: 0.8000


In [69]:
# Train the Scikit-learn decision tree
sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)
# Predict on the test set
y_pred_sklearn = sklearn_tree.predict(X_test)
# Calculate accuracy
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")

Scikit-learn Decision Tree Accuracy: 1.0000


In [70]:
print(f"Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")

Accuracy Comparison:
Custom Decision Tree: 0.8000
Scikit-learn Decision Tree: 1.0000


**task 3**

In [71]:
from sklearn.datasets import load_wine

# Load dataset
wine = load_wine()
X = wine.data      # features (13 columns)
y = wine.target    # target class (0,1,2)


In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


In [74]:
# Initialize Decision Tree
dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)

# Train the model
dt_model.fit(X_train, y_train)

# Predict on test set
y_pred_dt = dt_model.predict(X_test)

# Evaluate F1 score
f1_dt = f1_score(y_test, y_pred_dt, average='macro')
print(f"Decision Tree F1 Score: {f1_dt:.4f}")


Decision Tree F1 Score: 0.9432


In [75]:
# Initialize Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate F1 score
f1_rf = f1_score(y_test, y_pred_rf, average='macro')
print(f"Random Forest F1 Score: {f1_rf:.4f}")


Random Forest F1 Score: 1.0000


In [76]:
print(f"\nF1 Score Comparison:")
print(f"Decision Tree: {f1_dt:.4f}")
print(f"Random Forest: {f1_rf:.4f}")



F1 Score Comparison:
Decision Tree: 0.9432
Random Forest: 1.0000


In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define Random Forest
rf = RandomForestClassifier(random_state=42)

# Define hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [2, 3, 4, None],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           scoring='f1_macro',   # maximize F1 score
                           cv=5,                 # 5-fold cross-validation
                           n_jobs=-1)            # use all CPU cores

# Fit on training data
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Best F1 score from cross-validation
print("Best F1 Score (CV):", grid_search.best_score_)


Best Hyperparameters: {'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 100}
Best F1 Score (CV): 0.9788068209740036


In [78]:
#Train Decision Tree Regressor

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Select feature matrix X and target y
X_reg = X[:, 1:]   # all features except 'alcohol'
y_reg = X[:, 0]    # predict 'alcohol'

# Split into train/test
from sklearn.model_selection import train_test_split
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Train Decision Tree Regressor
dt_reg = DecisionTreeRegressor(max_depth=3, random_state=42)
dt_reg.fit(X_train_reg, y_train_reg)

# Predict and evaluate
y_pred_dt = dt_reg.predict(X_test_reg)
print("Decision Tree Regressor R2:", r2_score(y_test_reg, y_pred_dt))
print("Decision Tree Regressor MSE:", mean_squared_error(y_test_reg, y_pred_dt))


Decision Tree Regressor R2: 0.5120320319194656
Decision Tree Regressor MSE: 0.2913343474021956


In [79]:
#Train Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

# Train Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42)
rf_reg.fit(X_train_reg, y_train_reg)

# Predict and evaluate
y_pred_rf = rf_reg.predict(X_test_reg)
print("Random Forest Regressor R2:", r2_score(y_test_reg, y_pred_rf))
print("Random Forest Regressor MSE:", mean_squared_error(y_test_reg, y_pred_rf))


Random Forest Regressor R2: 0.7504662893678093
Random Forest Regressor MSE: 0.14898055917039113


In [80]:
#Hyperparameter Tuning using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)

# Define hyperparameter grid
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [2, 3, 4, None],
    'min_samples_split': [2, 5, 10]
}

# RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_reg,
                                   param_distributions=param_dist,
                                   n_iter=10,            # number of random combinations
                                   scoring='r2',         # maximize R2 score
                                   cv=5,                 # 5-fold CV
                                   random_state=42,
                                   n_jobs=-1)

# Fit the model
random_search.fit(X_train_reg, y_train_reg)

# Best parameters and score
print("Best Hyperparameters:", random_search.best_params_)
print("Best R2 Score (CV):", random_search.best_score_)


Best Hyperparameters: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 3}
Best R2 Score (CV): 0.5244990271923053
